In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
# Check if CUDA is available and set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda:0


In [30]:
# Import raw dataset to pandas dataframe
df_org = pd.read_csv('TMC_Report 6-current 1.csv')


In [31]:
df = df_org[["TESTER_ID","handler_id","product_no","QTY_IN","QTY_OUT","UPH"]]

In [32]:
df

,TESTER_ID,handler_id,product_no,QTY_IN,QTY_OUT,UPH
0,ADVT2000-10,NS8160-18,UTIC*TT3920D,10608,8234,2209.00
1,ADVT2000-10,NS8160-18,UTIC*TT3920D,448,416,504315.00
2,ADVT2000-12,EXCEED8016-52,4NB0*MY1UABA,0,0,0.00
3,ADVT2000-12,EXCEED8016-52,4NB0*MY1UABA,0,0,0.00
4,ADVT2000-12,EXCEED8016-52,4NB0*MY1UABA,0,0,0.00
...,...,...,...,...,...,...
172004,TMT-309,NX116-15,AMDG2730DN-J-GE4,18491,18004,3799.00
172005,TMT-46,NX116-16,AMDG2730DN-J-GE4,16181,15888,4280.00
172006,TMT-46,NX116-16,AMDG2730DN-J-GE4,25767,25722,4406.00
172007,TMT-96,NX116-27,AMDG2730DN-J-GE4,15922,15296,4685.45


In [33]:
df = df[df["UPH"] > 0]

In [34]:
df

,TESTER_ID,handler_id,product_no,QTY_IN,QTY_OUT,UPH
0,ADVT2000-10,NS8160-18,UTIC*TT3920D,10608,8234,2209.00
1,ADVT2000-10,NS8160-18,UTIC*TT3920D,448,416,504315.00
6,ADVT2000-13,EXCEED8016-07,UTIC*TT3920D,30,20,692308.00
7,CRAFT-U64-01,EG4090-W4-08,M29ADE9BSI0UT2910W0,23491,22986,33540.00
8,CRAFT-U64-01,EG4090-W4-08,M29ADE9BSI0UT2910W0,23490,23089,35147.00
...,...,...,...,...,...,...
172004,TMT-309,NX116-15,AMDG2730DN-J-GE4,18491,18004,3799.00
172005,TMT-46,NX116-16,AMDG2730DN-J-GE4,16181,15888,4280.00
172006,TMT-46,NX116-16,AMDG2730DN-J-GE4,25767,25722,4406.00
172007,TMT-96,NX116-27,AMDG2730DN-J-GE4,15922,15296,4685.45


In [35]:
# Input json for prediction 
#dfx = pd.read_json('x_input.json')  


In [36]:
# Separate features and target
X = df.drop('UPH', axis=1)
y = df['UPH']

In [37]:
X

,TESTER_ID,handler_id,product_no,QTY_IN,QTY_OUT
0,ADVT2000-10,NS8160-18,UTIC*TT3920D,10608,8234
1,ADVT2000-10,NS8160-18,UTIC*TT3920D,448,416
6,ADVT2000-13,EXCEED8016-07,UTIC*TT3920D,30,20
7,CRAFT-U64-01,EG4090-W4-08,M29ADE9BSI0UT2910W0,23491,22986
8,CRAFT-U64-01,EG4090-W4-08,M29ADE9BSI0UT2910W0,23490,23089
...,...,...,...,...,...
172004,TMT-309,NX116-15,AMDG2730DN-J-GE4,18491,18004
172005,TMT-46,NX116-16,AMDG2730DN-J-GE4,16181,15888
172006,TMT-46,NX116-16,AMDG2730DN-J-GE4,25767,25722
172007,TMT-96,NX116-27,AMDG2730DN-J-GE4,15922,15296


In [38]:
# using time module
import time

# ts stores the time in seconds
tstart = time.time()

# print the current timestamp
print(tstart)

1727684576.0673134


In [39]:
# Preprocess features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


In [40]:
# Fit and transform the features
X_processed = preprocessor.fit_transform(X)

In [41]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)


In [42]:
# Convert to PyTorch tensors and move to GPU
X_train = torch.FloatTensor(X_train.toarray()).to(device)
X_test = torch.FloatTensor(X_test.toarray()).to(device)
y_train = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
y_test = torch.FloatTensor(y_test.values).reshape(-1, 1).to(device)


In [43]:
X.shape

(163285, 5)

In [44]:
# Define the PyTorch model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc5 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.dropout(self.relu(self.bn1(self.fc1(x))))
        x = self.dropout(self.relu(self.bn2(self.fc2(x))))
        x = self.dropout(self.relu(self.bn3(self.fc3(x))))
        x = self.dropout(self.relu(self.bn4(self.fc4(x))))
        x = self.fc5(x)
        return x

In [45]:
# Initialize the model and move it to GPU
input_size = X_train.shape[1]
model = MLP(input_size).to(device)

In [46]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [47]:
# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [ ]:
# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        # Move batch to GPU
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/200], Loss: 887769792.0000
Epoch [20/200], Loss: 2408496128.0000
Epoch [30/200], Loss: 35948132.0000
Epoch [40/200], Loss: 18752048.0000
Epoch [50/200], Loss: 25660574.0000
Epoch [60/200], Loss: 10179033.0000
Epoch [70/200], Loss: 19901392.0000
Epoch [80/200], Loss: 169915280.0000
Epoch [90/200], Loss: 25492666.0000


In [ ]:
# Save the model
torch.save(model.state_dict(), 'testUPH_model.pth')

In [ ]:
# Predicting the results of the Test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test)

In [ ]:
# Move predictions back to CPU for numpy operations
y_pred_np = y_pred.cpu().numpy()
y_test_np = y_test.cpu().numpy()

In [ ]:
# Compute MAPE
MAPE = np.mean(100 * (np.abs(y_test_np - y_pred_np) / y_test_np))
print(f'Accuracy: {100 - MAPE:.2f}%')

In [ ]:
# Create comparison DataFrame
compare = pd.DataFrame({'y_test': y_test_np.flatten(), 'y_pred': y_pred_np.flatten()})
compare.to_csv('compare.csv', index=False)

In [ ]:
# Predict on the entire dataset
x_tensor = torch.FloatTensor(x).to(device)
model.eval()
with torch.no_grad():
    PRED = model(x_tensor)

In [ ]:
# Move predictions back to CPU and add to the original DataFrame
df['PRED'] = PRED.cpu().numpy().flatten()

# Export output to csv
df.to_csv('ballshear_regression.csv', index=False)